In [1]:
import numpy as np
import vtk
from vtk.numpy_interface import dataset_adapter as dsa
import glob
from vtk.util.numpy_support import vtk_to_numpy

In [3]:
working_directory = r'path\to\your\working\directory'
path_files = glob.glob(working_directory+'\\'+'centerline_X.vtp') 
print(path_files)

['C:\\CTEPH\\9A\\centerline_X.vtp']


In [4]:
for counter,filename in enumerate(path_files):
    # counter = the number of centerline file in the working directory that glob algorithm extracts

    reader = vtk.vtkXMLPolyDataReader()

    # Set the name of input file of the polyDataReader object
    reader.SetFileName(filename)
    reader.Update()

    # Get the data object from the polyDataReader object
    polydata = reader.GetOutput()

    pntsXYZ = dsa.WrapDataObject(polydata).Points

    # header = dsa.WrapDataObject(polydata).GetPointData().keys()

    MaxSphereRadius = dsa.WrapDataObject(polydata).GetPointData().GetArray(0)
    MaxSphereRadius = MaxSphereRadius.astype('float32')
    
    EdgeArray = dsa.WrapDataObject(polydata).GetPointData().GetArray(1)
    
    EdgePCoordArray = dsa.WrapDataObject(polydata).GetPointData().GetArray(2)
    
    ConArray_vtk = dsa.WrapDataObject(polydata).GetLines().GetConnectivityArray()
    ConnArray_np = vtk_to_numpy(ConArray_vtk) # datatype = numpy.ndarray
    # there are supposed to multiple rows of data (see paraview: toggle cell connectivity visibility)
    # multiple rows of data corresponds to multiple centerlines

    # Save the outputted csv in the specified working directory. Counter serves as an index for bookkeeping purpose
    np.savetxt(working_directory + '//'+'%d_centerline_pointsXYZ.csv' % counter, pntsXYZ)
    np.savetxt(working_directory + '//'+'%d_centerline_Max_Sphere_Radius.csv' % counter, MaxSphereRadius)
    np.savetxt(working_directory + '//'+'%d_centerline_Edge_Array.csv' % counter, EdgeArray)
    np.savetxt(working_directory + '//'+'%d_centerline_Connectivity.csv' % counter, ConnArray_np)
    